# 1.3 - Macrobond Data API for Python - Fetching multiple Time Series

This notebook is designed to act as a template and guidline in which certain elements can be minipulated to get the desired outcome. Here we demonstrates how you can use `get_series` to download multiple time series. 

The `get_many_series` function allows to fetch time series only when they have received updates compared to the last download. For this workflow, we will set datetime data parameter = LastModifiedTimeStamp.

The methods `get_series` and `get_many_series` have different use cases. 
The `get_series` function is convenient when you want to download a handfull of series to use in a model. You get an array back that you can index directly and as many times as you want.

The `get_many_series` function is useful when you want to synchronize data with your own database. It has the “LastModified” parameter for each series and it returns a “generator” rather than an array. A generator is not as convenient as an array for direct access, but it allows you to consume a large amount of data in a streaming fashion without loading all the data into memory before you can start processing the data.

You can find a full description of all methods and parameters used in the examples in the [documentation of the API](https://macrobond.github.io/macrobond-data-api/common/api.html).

*Full error handling is omitted for brevity*

***

## Importing packages

In [1]:
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import macrobond_data_api as mda
from datetime import datetime

***

## Get series with your universe

We are using `get_series` below. Enter your universe in the payload.

As an example, we use the close price of Copper's futures in different positions, from COMEX (1st to 24th)

In [2]:
entities = mda.get_series(
    "hg_c1_cl",
    "hg_c2_cl",
    "hg_c3_cl",
    "hg_c4_cl",
    "hg_c5_cl",
    "hg_c6_cl",
    "hg_c7_cl",
    "hg_c8_cl",
    "hg_c9_cl",
    "hg_c10_cl",
    "hg_c11_cl",
    "hg_c12_cl",
    "hg_c13_cl",
    "hg_c14_cl",
    "hg_c15_cl",
    "hg_c16_cl",
    "hg_c17_cl",
    "hg_c18_cl",
    "hg_c19_cl",
    "hg_c20_cl",
    "hg_c21_cl",
    "hg_c22_cl",
    "hg_c23_cl",
    "hg_c24_cl",
)

In [3]:
df = pd.json_normalize([x.to_dict() for x in entities])

In [4]:
df2 = (
    df.set_index(["metadata.Name", "metadata.FullDescription", "metadata.FuturesPosition"])[["Dates", "Values"]]
    .apply(pd.Series.explode)
    .reset_index()
)
df2

,metadata.Name,metadata.FullDescription,metadata.FuturesPosition,Dates,Values
0,hg_c1_cl,"Copper, COMEX, Future, COMEX, 1st Position, Cl...",c1,1959-07-02,0.293
1,hg_c1_cl,"Copper, COMEX, Future, COMEX, 1st Position, Cl...",c1,1959-07-06,0.291
2,hg_c1_cl,"Copper, COMEX, Future, COMEX, 1st Position, Cl...",c1,1959-07-07,0.282
3,hg_c1_cl,"Copper, COMEX, Future, COMEX, 1st Position, Cl...",c1,1959-07-08,0.2846
4,hg_c1_cl,"Copper, COMEX, Future, COMEX, 1st Position, Cl...",c1,1959-07-09,0.276
...,...,...,...,...,...
261469,hg_c24_cl,"Copper, COMEX, Future, COMEX, 24th Position, C...",c24,2023-04-12,4.0505
261470,hg_c24_cl,"Copper, COMEX, Future, COMEX, 24th Position, C...",c24,2023-04-13,4.084
261471,hg_c24_cl,"Copper, COMEX, Future, COMEX, 24th Position, C...",c24,2023-04-14,4.074
261472,hg_c24_cl,"Copper, COMEX, Future, COMEX, 24th Position, C...",c24,2023-04-17,4.0465


In [ ]:
df_26 = df2.loc[df2["Dates"] == "2022-09-26T00:00:00"]
df_27 = df2.loc[df2["Dates"] == "2022-09-27T00:00:00"]
df_28 = df2.loc[df2["Dates"] == "2022-09-28T00:00:00"]

In [ ]:
matplotlib.rcParams["figure.facecolor"] = "white"

plt.plot(df_26["metadata.FuturesPosition"], df_26["Values"], color="darkorange", marker="o", label="26th Septmber")
plt.plot(df_27["metadata.FuturesPosition"], df_27["Values"], color="green", marker="o", label="27th Septmber")
plt.plot(df_28["metadata.FuturesPosition"], df_28["Values"], color="midnightblue", marker="o", label="28th Septmber")
plt.title("Copper COMEX Future last 3 days", fontsize=14)
plt.xlabel("FuturesPosition")
plt.ylabel("Value")
plt.grid(True)
plt.legend(loc=2, ncol=2)
plt.autoscale(enable=True, axis="both", tight=None)
plt.show()

***

## Download only if series is updated

We now want to check whether our universe needs to be updated or not.
You can use the ifModifiedSince parameter within `get_many_series` method where it should correspond to the LastModifiedTimeStamp populated within the metadata of the initial load.

Please note that LastModifiedTimeStamp reflects not just changes in values, but can be just a change in the metadata.

In this example, as we are using time series with a daily frequency, it is most likely there will be updates from the LastModifiedTimeStamp.
In most cases, you do not want to include the series that are not modified in the result, but in this case we have set
the `include_not_modified` parameter to `True` in ordert to illustrate that you get a "Not modified" 304 response
using a time stamp as of the time of the refresh of this notebook.

The result will be in the same order as the request.

In [7]:
currentTime = datetime.now()
for i in mda.get_many_series(
    ("hg_c1_cl ", currentTime),
    ("hg_c2_cl ", currentTime),
    ("hg_c3_cl ", currentTime),
    ("hg_c4_cl ", currentTime),
    ("hg_c5_cl ", currentTime),
    ("hg_c6_cl ", currentTime),
    ("hg_c7_cl ", currentTime),
    ("hg_c8_cl ", currentTime),
    ("hg_c9_cl ", currentTime),
    ("hg_c10_cl", currentTime),
    ("hg_c11_cl", currentTime),
    ("hg_c12_cl", currentTime),
    ("hg_c13_cl", currentTime),
    ("hg_c14_cl", currentTime),
    ("hg_c15_cl", currentTime),
    ("hg_c16_cl", currentTime),
    ("hg_c17_cl", currentTime),
    ("hg_c18_cl", currentTime),
    ("hg_c19_cl", currentTime),
    ("hg_c20_cl", currentTime),
    ("hg_c21_cl", currentTime),
    ("hg_c22_cl", currentTime),
    ("hg_c23_cl", currentTime),
    ("hg_c24_cl", currentTime),
    include_not_modified = True
):
    if i.is_error:
        print(f"{i.name}:, {i.status_code}, {i.error_message}")

hg_c1_cl :, 304, Not modified
hg_c2_cl :, 304, Not modified
hg_c3_cl :, 304, Not modified
hg_c4_cl :, 304, Not modified
hg_c5_cl :, 304, Not modified
hg_c6_cl :, 304, Not modified
hg_c7_cl :, 304, Not modified
hg_c8_cl :, 304, Not modified
hg_c9_cl :, 304, Not modified
hg_c10_cl:, 304, Not modified
hg_c11_cl:, 304, Not modified
hg_c12_cl:, 304, Not modified
hg_c13_cl:, 304, Not modified
hg_c14_cl:, 304, Not modified
hg_c15_cl:, 304, Not modified
hg_c16_cl:, 304, Not modified
hg_c17_cl:, 304, Not modified
hg_c18_cl:, 304, Not modified
hg_c19_cl:, 304, Not modified
hg_c20_cl:, 304, Not modified
hg_c21_cl:, 304, Not modified
hg_c22_cl:, 304, Not modified
hg_c23_cl:, 304, Not modified
hg_c24_cl:, 304, Not modified


None of our time series have received any update. When there are updates, the time series will be downloaded by the request.

A typical workflow is to download the series the first time without specifying any timestamp, store the timestamp with the series, and then specify this in subsequent requests.

Here is below an example where we fetch just one series, without knowing the last modification time, and then we do a second request. In this case it is very unlikely that the series was updated between these calls, so the second request will most likely return status NOT_MODIFIED.

In [8]:
first = next(mda.get_many_series(("hg_c2_cl", None)))
last_modified = first.last_modified

print(f"First request: status: {first.status_code.name}, last_modified: {last_modified}")

# Here we imagine that we store this series and ask for updates at a later time

second = next(mda.get_many_series(("hg_c2_cl", last_modified, ), include_not_modified = True))
print(f"Second request: status: {second.status_code.name}")

First request: status: OK, last_modified: 2023-04-18 22:49:47+00:00
Second request: status: NOT_MODIFIED
